In [ ]:
# 패키지 import
import io
import os
import cv2
import json
import argparse
import imutils
import pickle
import numpy as np

from PIL import Image
from google.cloud import vision
from keras.models import load_model
from matplotlib import pyplot as plt
from keras_preprocessing.image import img_to_array

In [ ]:
# 이미지 경로
path ='/content/KakaoTalk_20230111_213307305-removebg-preview.png'

img = cv2.imread(path)

In [ ]:
# 이미지 전처리
image = cv2.resize(img, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# MultiLabelBinarizer 로드
model = load_model("/content/drive/MyDrive/pill_model.h5")
mlb = pickle.loads(open("/content/drive/MyDrive/multilabelbin.txt", "rb").read())

# 확률이 높은 상위 2개의 클래스 라벨 추출
proba = model.predict(image)[0]
idxs = np.argsort(proba)[::-1][:5]

1/1 [==============================] - 0s 174ms/step


In [ ]:
for (i, j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)

# 모든 라벨의 확률 출력
for (label, p) in zip(mlb.classes_, proba):
	print("{}: {:.2f}%".format(label, p * 100))

black: 0.08%
black,trans: 0.00%
blue: 0.01%
blue,light: 0.00%
blue,trans: 0.00%
bluishgreen: 0.04%
bluishgreen,trans: 0.01%
brown: 0.86%
brown,deep: 0.00%
brown,trans: 0.69%
circle: 0.07%
darkblue: 0.01%
darkblue,trans: 0.00%
diamond: 0.00%
ellipse: 7.51%
etc: 1.46%
gray: 0.09%
green: 0.77%
green,trans: 0.37%
orange: 1.00%
orange,trans: 0.22%
pink: 0.00%
pink,deep: 0.00%
pink,light: 0.00%
pink,trans: 0.00%
purple: 0.00%
purple,trans: 0.00%
rectangle: 89.92%
red: 0.18%
red,trans: 0.03%
trans: 3.78%
white: 0.04%
white,blue: 0.01%
white,brown: 0.00%
white,green: 0.00%
white,orange,trans: 0.00%
white,red: 0.00%
white,trans: 0.00%
white,yellow: 0.10%
wine: 0.02%
wine,trans: 0.00%
yellow: 5.83%
yellow,trans: 52.00%
yellowishgreen: 12.77%
yellowishgreen,trans: 0.12%


In [ ]:
# 확률이 높은 라벨 5개 출력
first_acc=mlb.classes_[idxs[0]]
second_acc=mlb.classes_[idxs[1]]
third_acc=mlb.classes_[idxs[2]]
fourth_acc=mlb.classes_[idxs[3]]
fifth_acc=mlb.classes_[idxs[4]]
print(first_acc)
print(second_acc)
print(third_acc)
print(fourth_acc)
print(fifth_acc)

rectangle
yellow,trans
yellowishgreen
ellipse
yellow


In [ ]:
# 알약의 모양과 색깔을 저장
tablet_shape=[]
tablet_color=[]

tablet_shape_labels=['circle','diamond','ellipse','rectangle','etc']
''''
tablet_color_labels=['black', 'black,trans', 'blue', 'blue,light', 'blue,trans', 'bluishgreen', 'bluishgreen,trans', 
'brown', 'brown,deep', 'brown,trans', 'darkblue', 'darkblue,trans', 'gray', 'green', 'green,trans', 
'orange', 'orange,trans', 'pink', 'pink,deep', 'pink,light', 'pink,trans', 'purple', 'purple,trans', 
'red', 'red,trans', 'trans', 'white', 'white,blue', 'white,brown','white,green', 'white,orange,trans', 
'white,red', 'white,trans', 'white,yellow', 'wine', 'wine,trans', 'yellow', 'yellow,trans', 'yellowishgreen', 'yellowishgreen,trans']
'''''
tablet_color_labels=['black','blue','blue,light','blue,trans','bluishgreen','bluishgreen,trans','brown',
' brown,trans','dark blue','dark blue','trans','gray','green','green,trans',
'orange','orange,trans','pink','pink,deep','pink,light','pink,trans','purple',
'purple,trans','red','red,trans','trans','white','white,blue','white,brown',
'white,green','white,red','white,trans','white,yellow','wine','wine,trans','yellow',
'yellow,trans','yellowishgreen','yellowishgreen,trans']

In [ ]:
for idx in range(5) :
    if mlb.classes_[idxs[idx]] in tablet_shape_labels :
        tablet_shape.append(mlb.classes_[idxs[idx]])
        
    elif mlb.classes_[idxs[idx]] in tablet_color_labels :
        tablet_color.append(mlb.classes_[idxs[idx]])
        

In [ ]:
# Google Vision API 사용
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='/content/drive/MyDrive/VisionAPI/seraphic-rarity-373904-e25d9c3b1870.json'

client = vision.ImageAnnotatorClient()

with io.open('/content/KakaoTalk_20230111_213307305-removebg-preview.png', 'rb') as image_file:    
    content = image_file.read()
    
image = vision.Image(content=content)

response = client.text_detection(image=image)
texts = response.text_annotations

# 찾은 글자 모두 저장
search_word=texts[1].description +' '+texts[2].description

In [ ]:
search_word

'YH DXP'

In [ ]:
import pandas as pd
df1=pd.read_csv('/content/test.csv',encoding='cp949')

In [ ]:
#dataframe 인덱싱
check1 = df1[(df1['의약품제형']==tablet_shape[0]) & (df1['색상앞']==tablet_color[0])]  
check2 = check1[(check1['표시앞']==search_word) | (check1['표시뒤']==search_word)]

In [ ]:
check2

,품목명,품목일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,색상뒤


In [ ]:
# datafeame을 json으로 쓰기
check2.to_json('/content/test.json', orient='columns')

with open('/content/test.json') as json_file:
    json_data = json.load(json_file)

In [ ]:
json_data

{'품목명': {},
 '품목일련번호': {},
 '업소명': {},
 '성상': {},
 '큰제품이미지': {},
 '표시앞': {},
 '표시뒤': {},
 '의약품제형': {},
 '색상앞': {},
 '색상뒤': {}}

In [ ]:
tablet_name = json_data["품목명"]

In [ ]:
if len(tablet_name) != 0:
  print(tablet_name)

if len(tablet_name) == 0:
  print("알약을 찾을 수 없습니다!")

알약을 찾을 수 없습니다!
